# Regresion Lineal Múltiple

# Maestría en Ciencia de Datos

## Francisco ME

---

### Cargar librerias

Se cargan las librerias que son necesarias para las funciones dentro del proyecto

In [5]:
import numpy as np                                        # tratamiento cientifico de numeros
import pandas as pd                                       # dataframes
import statsmodels.api as sm                              # modelos estadisticos
from datetime import datetime                             # tratamiento de datetime
from functools import reduce                              # para la union de los dataframes
from os import listdir, getcwd                            # para leer todos los archivos de un folder
from os.path import isfile, join                          # encontrar y unir archivos en un folder

from sklearn import linear_model                          # modelo lineal
from sklearn.model_selection import train_test_split      # separacion de conjunto de entrenamiento y prueba
from statsmodels.formula.api import ols                   # ols

### Cargar las deependencias

Se cargan las dependencias del proyecto, en este caso son las siguientes:

- **visualizaciones.py** : Donde se encuentran funciones que fueron especialmente ddesarrolladas para visualizar datos en este proyecto.

- **datos.py** : En el cual realiza la función fundamental de contener los parámetros y _token_ para utilizar el servicio de *API* de *OANDA* y solicitar precios históricos, que son regresados en el objeto *pd_hist* cargado finalmente como *precios*. *OANDA* es un *broker* que provee acceso programático a mercados financieros bursátiles de USA, este *broker* convenientemente provee de una **API** o Interfáz Programática de Aplicaciones (por sus siglas en inglés), esto de manera gratuita y con extensa documentación sobre su uso con *python* (mas información al respecto puede ser consultada en su sitio [developer.oanda.com]('http://developer.oanda.com/')

Ambos códigos puede ser encontrardos dentro de la misma carpeta de este proyecto

In [7]:
import rlm.visualizaciones as vs                              # script con funciones para visualizar
from rlm.datos import pd_hist as precios                      # importar precios

ModuleNotFoundError: No module named 'rlm'

### Opciones para visualizacion de DataFrame

Se establecen las siguientes opciones para visualizar dataframes en la consola de una manera mas libre

In [ ]:
pd.set_option('display.max_rows', None)                   # sin limite de renglones maximos para mostrar pandas
pd.set_option('display.max_columns', None)                # sin limite de columnas maximas para mostrar pandas
pd.set_option('display.width', None)                      # sin limite el ancho del display
pd.set_option('display.expand_frame_repr', False)         # visualizar todas las columnas de un pandas dataframe
pd.options.mode.chained_assignment = None                 # para evitar el warning enfadoso de indexacion

## Planteamiento de la problematica y contexto

Se plantea el reto de modelar el precio semanal del tipo de cambio _Euro Vs Dólar Americano_ , representado por el instrumento financiero de tipo de cambio cruzado con el ticker _EurUsd_. 

- La variable dependiente: **Close** - Precio de cierre semanal del tipo de cambio *EurUsd*
- Las variables independientes o explicativas: **Indicadores macroeconómicos semanales** de las economías de USA y la Zona Euro.

### Calendario Económico

In [4]:
# Obtener el director donde se encuentran todos los archivos
directorio = getcwd() + '/archivos/'

# Obtener una lista de todos los archivos de datos
archivos_ce = [f for f in listdir(directorio) if isfile(join(directorio, f))]


def f_datos_ce(p0_ind):

    # Datos de entrada
    datos = pd.read_csv('archivos/' + p0_ind)

    # Convertir columna de fechas a datetime
    datos['DateTime'] = [datetime.strptime(datos['DateTime'][i], '%m/%d/%Y %H:%M:%S')
                         for i in range(0, len(datos['DateTime']))]

    # Ordernarlos de forma ascendente en el tiempo
    datos.sort_values(by=['DateTime'], inplace=True, ascending=True)
    datos = datos.reset_index(drop=True)

    # Agregar columna con numero de año y semana (Servira para empatar fecha de indicador con fecha de precio)
    datos['Year_Week'] = [datos.loc[i, 'DateTime'].strftime("%Y") + '_' +
                          datos.loc[i, 'DateTime'].strftime("%W") for i in range(0, len(datos['DateTime']))]

    # Solo tomar las columnas para el ejercicio
    datos = datos[['Year_Week', 'Actual']]

    print(datos.head())

    return datos


# Unir dataframes
dataframes = [f_datos_ce(p0_ind=archivos_ce[i]) for i in range(0, len(archivos_ce))]
datos_ce = reduce(lambda left, right: pd.merge(left, right, on=['Year_Week'], how='inner'), dataframes)

# Renombrar las columnas
variables_ce = ['x_' + str(i) for i in range(0, len(archivos_ce))]
datos_ce.columns = ['Year_Week'] + variables_ce

NameError: name 'getcwd' is not defined

### Precios de OANDA

In [ ]:
# Generacion de variables explicativas
datos_fx = precios[['TimeStamp', 'Close', 'Open']]

# Convertir a valor tipo float
datos_fx.loc[:, 'Close'] = datos_fx.loc[:, 'Close'].astype(float)
datos_fx.loc[:, 'Open'] = datos_fx.loc[:, 'Open'].astype(float)

# Calcular el rendimiento logaritmico como variable a explicar Y
datos_fx.loc[:, 'log_ret'] = np.log(datos_fx.loc[:, 'Close']/datos_fx.loc[:, 'Close'].shift(1))

# Calcular la diferencia entre precio de cierre y apertura multiplicado por el multiplicador (PIP)
datos_fx.loc[:, 'pips_co'] = (datos_fx.loc[:, 'Close'] - datos_fx.loc[:, 'Open'])*10000

# Eliminar el NaN inicial
datos_fx = datos_fx.dropna()

# Resetear index de data frame
datos_fx = datos_fx.reset_index(drop=True)

# Convertir columna de fechas a datetime
datos_fx['TimeStamp'] = pd.to_datetime(datos_fx['TimeStamp'])

# Agregar columna de semana
# datos_fx['Semana'] = [datos_fx.loc[i, 'TimeStamp'].strftime("%W") for i in range(0, len(datos_fx['TimeStamp']))]
datos_fx['Year_Week'] = [datos_fx.loc[i, 'TimeStamp'].strftime("%Y") + '_' +
                         datos_fx.loc[i, 'TimeStamp'].strftime("%W") for i in range(0, len(datos_fx['TimeStamp']))]

### Union de datos en un solo DataFrame

In [ ]:
# unir los dos dataframes para obtener el dataframe a usar en el modelo
datos_md = pd.merge(datos_fx, datos_ce, on=['Year_Week'], how='inner')

# eliminar la columna Year_Week
datos_md = datos_md.drop(['Year_Week'], axis=1)

### Exploracion de variables

In [ ]:
# Matriz de diagramas de dispersion
g1 = vs.explora(datos_md)